In [1]:
import pandas as pd
import numpy as np

# read in all files from the folder called ../data
heart_table = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022.csv')
medical_source = pd.read_csv('../data/CF_medicalData.csv')
df_vaprediction_v1 = pd.read_csv('../data/cf_vaprediction_v1.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_11684\2792565412.py:6: DtypeWarning: Columns (16,17,18,19,20,21,22,23,34,36,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,74,75,76,78,79,80,85,87,88,89,90,91,92,93,97,98,99,108,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  medical_source = pd.read_csv('../data/CF_medicalData.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_11684\2792565412.py:7: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,25,26,27,29,30,31,32,33,34,35,39,41,42,43,44,45,46,47,48,49,51,52,53,55,56,57,58,59,60,61,62,65,66,67,68,69,70,72,78,83,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vaprediction_v1 = pd.read_csv('../data/cf_vaprediction_v1.csv')


In [2]:
# in the table heart_table, the column called 'source_id' grab rows with substring 'V1' and store in a new dataframe
df_v1 = heart_table[heart_table['source_id'].str.contains('V1')]

In [3]:
# how many diplicates are in v1_pop when just looking at first name, last name
df_v1.duplicated(subset=['FIRST_NAME', 'LAST_NAME']).sum()
#store the duplicated rows in a new dataframe
df_v1_dup = df_v1[df_v1.duplicated(subset=['FIRST_NAME', 'LAST_NAME'])]

In [4]:
#make a subset of heart_table where the column source_id contains 'V1_population2_sample'
v1_pop = heart_table[heart_table['source_id'].str.contains('V1_population2_sample')]

v1_pop = v1_pop[['source_id','FIRST_NAME', 'LAST_NAME', 'MASTER_DOB']]

# # merge v1_pop with medical_source on the columns FIRST_NAME, LAST_NAME, and DOB
# v1_pop_medical = pd.merge(medical_source, v1_pop, right_on=['FIRST_NAME', 'LAST_NAME', 'MASTER_DOB'], left_on=['FirstName', 'LastName', 'BirthDate'], how='left')

# merge v1_pop with medical_source on the columns FIRST_NAME, LAST_NAME, and DOB
v1_pop_medical = pd.merge(medical_source, v1_pop, right_on=['FIRST_NAME', 'LAST_NAME'], left_on=['FirstName', 'LastName'], how='left')


In [5]:
#drop columns that are not needed in the new dataframe
v1_pop_medical = v1_pop_medical.drop(columns=['FIRST_NAME', 'LAST_NAME', 'MASTER_DOB'])

# if the column 'source_id_x' is null, then replace with the value in the column 'source_id_y'
v1_pop_medical['source_id_x'] = v1_pop_medical['source_id_x'].fillna(v1_pop_medical['source_id_y'])
# drop the column 'source_id_y'
v1_pop_medical = v1_pop_medical.drop(columns=['source_id_y'])
# rename the column 'source_id_x' to 'source_id'
v1_pop_medical = v1_pop_medical.rename(columns={'source_id_x': 'source_id'})

In [6]:
#count the number of rows in the column 'source_id' that are not null
print(medical_source['source_id'].count())
#count the number of rows in the column 'source_id' that are not null
print(v1_pop_medical['source_id'].count())

11193
12312


In [7]:
#print how many rows are in the dataframe in column 'source_id' that are null
print(v1_pop_medical['source_id'].isnull().sum())

327


487

In [8]:
# create a subset of the dataframe where the column 'source_id' is not null
v1_pop_medical_sub = v1_pop_medical[v1_pop_medical['source_id'].notnull()]
# check to see if there are any duplicates in the dataframe
v1_pop_medical_sub.duplicated(subset=['source_id']).sum()
# create a subset of the dataframe with all the duplicates
v1_pop_medical_sub_dup = v1_pop_medical_sub[v1_pop_medical_sub.duplicated(subset=['source_id'])]



In [9]:
# write the dataframe to a csv file
v1_pop_medical.to_csv('../data/out/CF_medicalData_v2.csv', index=False)